# Phindr3D in python

This notebook acts as a basic GUI/framework for Phindr3D to work in python.

In [54]:
import phindr_functions as phi
import phindr_organoidCSApp as org
import phindr_clustering as clu
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob

# Preliminary set-up:

In [55]:
param = phi.initParameters()

#change parameters as needed
param.tileX = 10
param.tileY = 10
param.tileZ = 3
param.intensityThresholdTuningFactor = 0.5
param.numVoxelBins = 20
param.numSuperVoxelBins = 15
param.numMegaVoxelBins = 40
param.minQuantileScaling = .5
param.maxQuantileScaling = .5
param.randTrainingSuperVoxel = 10000
param.superVoxelThresholdTuningFactor = 0.5
param.megaVoxelTileX = 5
param.megaVoxelTileY = 5
param.megaVoxelTileZ = 2
param.countBackground = False
param.megaVoxelThresholdTuningFactor = 0.5
param.pixelsPerImage = 200 
param.randTrainingPerTreatment = 1
param.randTrainingFields = 5
param.showImage = 0
param.startZPlane = 1
param.endZPlane = 500
param.numRemoveZStart = 1
param.numRemoveZEnd = 1
param.computeTAS = 0
param.trainingPerColumn = False
param.intensityNormPerTreatment = False
param.treatmentColNameForNormalization = ''
param.imageTreatments = []
param.allTreatments = []
param.trainingColforImageCategories = []
param.superVoxelPerField = param.randTrainingSuperVoxel//param.randTrainingFields
param.lowerbound = [0, 0, 0]
param.upperbound = [1, 1, 1]
param.numChannels = 3 

#don't need to touch these 
param.svcolormap = phi.random_cmap(map_len=param.numSuperVoxelBins+1)
param.mvcolormap = phi.random_cmap(map_len=param.numMegaVoxelBins+1)


# File loading:
Load folder containing images of interest

NOTE:
-  image names must follow strucuture: (image identification code) + p(z stack number) + ch(channelnumber) + (whatever) + .tiff 
- images in folder also need to have consistent dimensions (constant number of z slices; constant number of channels)

In [60]:
folder_path = r'D:\20210326 - iN_BH3Mimetic_Phindr3D\20210326 - iN_BH3-mimetic_Phindr3D__2021-03-26T07_48_34-Measurement 1\Images' #r'path_to_images'

files, imageIDs, treatmentids, idstreatment = phi.get_files(folder_path, ID_pos='start', ID_mark=None, treat_mark='r', treat_endmark='f', ID_markextra=None, slice_mark='p0', chan_mark='ch')

#will be empty list if treat_mark is none.
param.imageTreatments = idstreatment
param.treatmentColNameForNormalization = treatmentids
if len(idstreatment) > 0:
    param.allTreatments = np.array(list(treatmentids.keys()), dtype='object')

#change This parameter below to True to get scaling factors separately for each treatment.
param.intensityNormPerTreatment = False

print('number of images:', len(imageIDs))
print(param.allTreatments)
#properly calculate number of channels
#slices in first ID:
tmpslices = list(files[imageIDs[0]].keys())
param.numChannels = len(files[imageIDs[0]][tmpslices[0]])

number of images: 540
[]


# Rescale intensities:

want to rescale to between 1 and 0

In [61]:
allImageID = imageIDs
param = phi.getScalingFactorforImages(files, allImageID, param)

print('Lowerbounds:', param.lowerbound)
print('Upperbounds:', param.upperbound)


Lowerbounds: [ 99.08 116.   104.04 100.08]
Upperbounds: [7567.96   1865.56   8964.3296 8655.    ]


# Threshold images:
get threshold value to use from training data


In [62]:
param = phi.getImageThresholdValues(files, allImageID, param)
intensityThreshold = np.quantile(param.intensityThresholdValues, param.intensityThresholdTuningFactor, axis=0)
param.intensityThreshold = np.reshape(intensityThreshold, (1, param.numChannels))

print(param.intensityThreshold)

[[0.20592893 0.06924598 0.1179663  0.23632812]]


# Cluster pixels:
compute pixel categories 

In [63]:
param = phi.getPixelBinCenters(files, allImageID, param)
print(param.pixelBinCenters) #20 different pixel categories made up of scaled intensities of different channels
print(param.pixelBinCenters.shape)

[[0.26512721 0.01670569 0.16763494 0.01680439]
 [0.5726921  0.04905906 0.05621715 0.3564156 ]
 [0.54507668 0.03338202 0.47292672 0.10127739]
 [0.1232063  0.06520851 0.01433494 0.94934756]
 [0.41766369 0.17119733 0.01485843 0.0221358 ]
 [0.84282168 0.07223079 0.17127184 0.1071016 ]
 [0.25147396 0.03341176 0.54983311 0.02851225]
 [0.01654579 0.10398433 0.01268846 0.58701842]
 [0.17538373 0.80916345 0.08937054 0.01979731]
 [0.90039864 0.04263686 0.03599882 0.63178337]
 [0.01070331 0.09709273 0.02679802 0.32376061]
 [0.39728403 0.02105368 0.16995507 0.02329171]
 [0.21820242 0.08428493 0.90982547 0.0671315 ]
 [0.06927865 0.15216524 0.18957105 0.01923234]
 [0.28077701 0.03028255 0.04358195 0.31457185]
 [0.13776095 0.94982739 0.01370836 0.77162265]
 [0.29781767 0.02007975 0.32413986 0.02219474]
 [0.38585705 0.02339636 0.03674673 0.53033031]
 [0.25797774 0.10390292 0.00787031 0.01648369]
 [0.75431926 0.0403799  0.86352441 0.05240631]]
(20, 4)


# Define super voxels:
use pixel categories to make some supervoxels! (also cluster super voxels into bins/categories)

In [64]:
# param.showImage = True

param = phi.getSuperVoxelBinCenters(files, allImageID, param)
print(param.supervoxelBincenters) #15 different supervoxel categories made up of frequencies of 20 different pixel categories
print(param.supervoxelBincenters.shape)
param.showImage = False


c:\Users\teole\anaconda3\envs\phy479\phindr_functions.py:1347: RuntimeWarning: invalid value encountered in true_divide
  superVoxelProfile = np.divide(superVoxelProfile, np.array([np.sum(superVoxelProfile, axis=1)]).T) #dont worry about divide by zero errors, they are supposed to happen here!


[[2.76647203e-04 4.51028104e-17 1.04083409e-17 1.07002361e-01
  1.08323584e-05 2.08166817e-17 1.89173608e-03 2.67003142e-01
  6.42500613e-04 1.21430643e-17 5.69733503e-01 1.04083409e-16
  7.61404044e-04 3.68426633e-02 1.06814002e-02 2.24377533e-04
  7.23633276e-04 6.93918224e-05 4.13640788e-03 0.00000000e+00]
 [1.79873222e-02 1.94594949e-05 8.12033690e-06 5.32407231e-05
  1.80809549e-02 8.53979801e-06 9.22750115e-05 5.40413641e-05
  3.87039744e-04 1.38777878e-17 5.10975123e-04 1.25035713e-02
  9.63303697e-06 1.16019738e-02 5.95040458e-03 1.73472348e-18
  5.98699089e-04 7.85333868e-05 9.32055216e-01 6.07153217e-18]
 [1.07606874e-03 6.93889390e-17 1.82145965e-17 1.38777878e-17
  4.38212095e-06 1.45716772e-16 5.76315578e-03 1.86503803e-05
  4.37008918e-03 3.12250226e-17 3.58761115e-03 1.80411242e-16
  5.87168766e-04 9.74072607e-01 1.14491749e-16 1.73472348e-18
  1.76350484e-03 7.63278329e-17 8.75676230e-03 3.03576608e-18]
 [8.45515623e-03 2.60513270e-03 1.16932409e-04 3.72822880e-04
  3.6

# Combine super voxels to mega voxels:
use super voxel categories and tileprofile to get megavoxels. also get megavoxel bins/categories

In [65]:
param = phi.getMegaVoxelBinCenters(files, allImageID, param)
print(param.megaVoxelBincenters)
print(param.megaVoxelBincenters.shape)

c:\Users\teole\anaconda3\envs\phy479\phindr_functions.py:1347: RuntimeWarning: invalid value encountered in true_divide
  superVoxelProfile = np.divide(superVoxelProfile, np.array([np.sum(superVoxelProfile, axis=1)]).T) #dont worry about divide by zero errors, they are supposed to happen here!
c:\Users\teole\anaconda3\envs\phy479\phindr_functions.py:922: RuntimeWarning: invalid value encountered in true_divide
  megaVoxelProfile = np.divide(megaVoxelProfile, np.array([np.sum(megaVoxelProfile, axis=1)]).T) #hopefully this works, they ask for elementwise, but the arrays seem to have different shapes.


[[3.88888889e-03 1.44140900e-02 2.50000000e-03 2.27189125e-02
  2.10647994e-02 1.06377551e-02 4.57249233e-01 3.28560224e-02
  9.27490788e-03 2.87055979e-01 6.66231961e-03 8.72046541e-02
  7.71739130e-03 2.79975900e-02 8.75745711e-03]
 [1.53717825e-02 2.99488640e-02 3.34329754e-01 2.39145851e-02
  3.48144461e-02 0.00000000e+00 6.93889390e-18 2.11239472e-02
  1.28949065e-03 1.01772122e-02 5.38801109e-03 1.28949065e-03
  2.61724077e-01 2.60628340e-01 6.93889390e-18]
 [7.05760997e-03 2.55725416e-02 1.66043720e-02 6.00306520e-02
  1.97727273e-02 1.93851981e-02 2.44765984e-01 1.83055729e-01
  1.04845229e-02 7.91312803e-02 4.28051116e-02 1.00042950e-01
  3.50710217e-02 1.54921598e-01 1.29870130e-03]
 [8.54700855e-04 2.01564081e-02 2.84651246e-03 1.66273375e-02
  1.45614035e-02 3.07829007e-02 1.08144621e-01 2.08862213e-02
  6.41025641e-03 7.19903244e-02 6.93877551e-03 2.90813602e-01
  1.30569599e-02 2.29285245e-02 3.73001452e-01]
 [8.06978604e-01 3.53441040e-03 1.42253668e-02 1.27713921e-03
  

# Profile images:


In [66]:
# to use previously calculated categories, uncomment lines below to specify different location of files 
# to be analyzed: EDIT HERE

# folder_path =  r'E:\Phindr3D_screenC\screenC\plate1' 

# files, imageIDs = phi.get_files(folder_path, ID_pos='start', ID_mark=None, ID_markextra='OB\d', slice_mark='_Z', chan_mark='_CH')

#Set to True to show images in process (will show LARGE number of images): EDIT HERE
param.showImage = False

#customize output file and location: EDIT HERE
outputFolder = r''
outputFileName ='jamesneuron.csv'

param, resultIM, resultRaw, df = phi.extractImageLevelTextureFeatures(files, allImageID, param, outputFileName=outputFileName, outputDir=outputFolder)
df

c:\Users\teole\anaconda3\envs\phy479\phindr_functions.py:1347: RuntimeWarning: invalid value encountered in true_divide
  superVoxelProfile = np.divide(superVoxelProfile, np.array([np.sum(superVoxelProfile, axis=1)]).T) #dont worry about divide by zero errors, they are supposed to happen here!
c:\Users\teole\anaconda3\envs\phy479\phindr_functions.py:922: RuntimeWarning: invalid value encountered in true_divide
  megaVoxelProfile = np.divide(megaVoxelProfile, np.array([np.sum(megaVoxelProfile, axis=1)]).T) #hopefully this works, they ask for elementwise, but the arrays seem to have different shapes.
c:\Users\teole\anaconda3\envs\phy479\phindr_functions.py:654: RuntimeWarning: invalid value encountered in true_divide
  imageProfile = imageProfile / np.sum(imageProfile) #normalize the image profile



All done.


,ImageID,Treatment,numMV,MV1,MV2,MV3,MV4,MV5,MV6,MV7,...,MV31,MV32,MV33,MV34,MV35,MV36,MV37,MV38,MV39,MV40
0,pl2r05c02f01,N/A,683.0,0.0,0.000000,0.033675,0.000000,0.000000,0.026354,0.052709,...,0.130307,0.019034,0.040996,0.020498,0.011713,0.030747,0.000000,0.062958,0.001464,0.024890
1,pl2r05c02f02,N/A,921.0,0.0,0.000000,0.022801,0.000000,0.000000,0.015201,0.059718,...,0.122693,0.029316,0.041260,0.035831,0.007600,0.048860,0.000000,0.048860,0.000000,0.061889
2,pl2r05c02f03,N/A,489.0,0.0,0.000000,0.040900,0.000000,0.000000,0.034765,0.120654,...,0.124744,0.012270,0.024540,0.012270,0.008180,0.030675,0.000000,0.083845,0.002045,0.044990
3,pl2r05c02f04,N/A,536.0,0.0,0.000000,0.027985,0.000000,0.000000,0.016791,0.059701,...,0.171642,0.009328,0.044776,0.024254,0.022388,0.020522,0.000000,0.057836,0.000000,0.065299
4,pl2r05c02f05,N/A,288.0,0.0,0.000000,0.048611,0.000000,0.000000,0.027778,0.107639,...,0.163194,0.000000,0.027778,0.024306,0.034722,0.031250,0.000000,0.027778,0.000000,0.045139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,pl2r07c13f11,N/A,502.0,0.0,0.037849,0.033865,0.007968,0.005976,0.027888,0.003984,...,0.027888,0.009960,0.015936,0.025896,0.019920,0.047809,0.011952,0.007968,0.000000,0.021912
536,pl2r07c13f12,N/A,900.0,0.0,0.041111,0.014444,0.015556,0.024444,0.003333,0.006667,...,0.010000,0.035556,0.006667,0.025556,0.020000,0.037778,0.024444,0.000000,0.003333,0.041111
537,pl2r07c13f13,N/A,461.0,0.0,0.004338,0.008677,0.008677,0.013015,0.019523,0.002169,...,0.013015,0.034707,0.006508,0.041215,0.028200,0.058568,0.000000,0.006508,0.010846,0.069414
538,pl2r07c13f14,N/A,75.0,0.0,0.000000,0.000000,0.000000,0.026667,0.000000,0.080000,...,0.040000,0.013333,0.066667,0.026667,0.013333,0.040000,0.000000,0.026667,0.000000,0.013333


In [67]:
import pickle
with open('paramfile.pkl', 'wb') as f:
    pickle.dump(param, f, pickle.HIGHEST_PROTOCOL)


# Cluster images:

# Classify images: